In [6]:
import os
import openai
from dotenv import load_dotenv, find_dotenv
from langchain.vectorstores import FAISS
# from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.embeddings import OpenAIEmbeddings
import pandas as pd

load_dotenv(find_dotenv())
openai.api_key = os.environ['OPENAI_API_KEY']

In [7]:
model_name = 'text-embedding-ada-002'
embed = OpenAIEmbeddings(
    model=model_name,
)

vectorstore = FAISS.load_local("arxiv_vectorstore", embeddings=embed)

In [18]:
query = "how can BERT be trained to optimize the number of parameters used?"

# Similarity search
test_search = vectorstore.similarity_search_with_score(
    query,  # our search query
    k=3  # return 3 most relevant docs
)

# How to extract text content and metadata from the first result returned by the similarity search
first_chunk = test_search[0][0].page_content
first_metadata = test_search[0][0].metadata

# Papers from where the first 3 chunks were taken out of
for doc in test_search:
    print(doc[0].metadata['title'])

ALBERT: A Lite BERT for Self-supervised Learning of Language Representations
ALBERT: A Lite BERT for Self-supervised Learning of Language Representations
RoBERTa: A Robustly Optimized BERT Pretraining Approach


In [23]:
query = "what research has been done in the field of LLM tool usage?"

test_search2 = vectorstore.similarity_search_with_score(
    query,  # our search query
    k=3  # return 3 most relevant docs
)

# Papers from where the first 3 chunks were taken out of
for doc in test_search2:
    print(doc[0].metadata['title'])

Large Language Models as Tool Makers
Gorilla: Large Language Model Connected with Massive APIs
A Survey of Large Language Models
